<a href="https://colab.research.google.com/github/RoelPi/name_analysis/blob/master/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
df_f = pd.read_excel('https://statbel.fgov.be/sites/default/files/files/documents/bevolking/5.10%20Namen%20en%20voornamen/5.10.%203%20Voornamen%20meisjes%20en%20jongens/TA_BIRTHS_FEMALE_2009_2018.xlsx')

df_m = pd.read_excel('https://statbel.fgov.be/sites/default/files/files/documents/bevolking/5.10%20Namen%20en%20voornamen/5.10.%203%20Voornamen%20meisjes%20en%20jongens/TA_BIRTHS_MALE_2009_2018.xlsx')

df_all_m = pd.read_excel('https://statbel.fgov.be/sites/default/files/files/documents/bevolking/5.10%20Namen%20en%20voornamen/5.10.%203%20Voornamen%20meisjes%20en%20jongens/Firstnames_birth_1995-.xlsx', sheet_name = 0)

df_all_f = pd.read_excel('https://statbel.fgov.be/sites/default/files/files/documents/bevolking/5.10%20Namen%20en%20voornamen/5.10.%203%20Voornamen%20meisjes%20en%20jongens/Firstnames_birth_1995-.xlsx', sheet_name = 1)

In [0]:
df_all_m = df_all_m.iloc[:,[4,5]]
df_all_f = df_all_f.iloc[:,[4,5]]

df_all_m.columns = ['naam','aantal']
df_all_m['geslacht'] = 'm'

df_all_f.columns = ['naam','aantal']
df_all_f['geslacht'] = 'v'

df_all = df_all_m.append(df_all_f)
df_all = df_all.set_index('naam')

In [0]:
df_all = df_all.sort_values('aantal', ascending = False)
df_all = df_all.dropna(axis = 0, how = 'any')

In [39]:
df_all.head()

,aantal,geslacht
naam,,
Emma,8747.0,v
Julie,8012.0,v
Laura,8010.0,v
Marie,7694.0,v
Thomas,7514.0,m


In [59]:
df_all['eerste_letter'] = df_all.index.str[0:1].str.upper()
df_all['laatste_letter'] = df_all.index.str[-1:].str.upper()
df_all['aantal_tekens'] = df_all.index.str.len()
df_all['aantal_tekens']

naam
Emma           4
Julie          5
Laura          5
Marie          5
Thomas         6
              ..
Peppe          5
Pawel          5
Paul-Henri    10
Pasquinel      9
Éléonore       8
Name: aantal_tekens, Length: 14917, dtype: int64